In [159]:
import requests
import pandas as pd



# Make the request with the appropriate headers
response = requests.get('https://datausa.io/api/data?drilldowns=Nation&measures=Population')

# Check if the request was successful
if response.status_code == 200:
    # Parse and use the response data
    data = response.json()

    data_values = data['data']

    # Convert the extracted data into a pandas DataFrame
    df_pop = pd.DataFrame(data_values)

    # Drop the specified columns
    columns_to_drop = ['ID Nation', 'ID Year', 'Slug Nation']
    df_pop = df_pop.drop(columns=columns_to_drop)

    # Print the DataFrame
    print(df_pop)
else:
    print(f'Error: {response.status_code} - {response.text}')



          Nation  Year  Population
0  United States  2021   329725481
1  United States  2020   326569308
2  United States  2019   324697795
3  United States  2018   322903030
4  United States  2017   321004407
5  United States  2016   318558162
6  United States  2015   316515021
7  United States  2014   314107084
8  United States  2013   311536594


In [160]:
#Try to read the file
try:
    # Read the CSV file
    df = pd.read_csv("API_GC.DOD.TOTL.GD.ZS_DS2_en_csv_v2_44281.csv", skiprows=4)

    # Filter rows for the United States
    us_data = df[df['Country Name'] == 'United States']

    # Melt the DataFrame to make years rows
    us_data = us_data.melt(id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
                                   var_name='Year',
                                   value_name='Value')

    #clean the data
    columns_to_drop2 = ['Country Code', 'Indicator Name', 'Indicator Code']
    us_data = us_data.drop(columns=columns_to_drop2)
    us_data = us_data.dropna(subset=['Value'])
    # Print the transformed DataFrame
    print(us_data)

except Exception as e: #error handling
    print(f"Error reading CSV file: {e}")


     Country Name  Year       Value
29  United States  1989   39.128400
30  United States  1990   40.933944
31  United States  1991   44.061597
32  United States  1992   46.050144
33  United States  1993   48.246140
34  United States  1994   47.353482
35  United States  1995   47.209535
36  United States  1996   46.741769
37  United States  1997   44.117470
38  United States  1998   41.129149
39  United States  1999   37.727278
40  United States  2000   33.267655
41  United States  2001   52.443676
42  United States  2002   53.862514
43  United States  2003   56.250405
44  United States  2004   56.662286
45  United States  2005   56.538848
46  United States  2006   55.467336
47  United States  2007   55.659926
48  United States  2008   63.815130
49  United States  2009   75.842048
50  United States  2010   84.964411
51  United States  2011   89.546817
52  United States  2012   93.649262
53  United States  2013   95.534688
54  United States  2014   95.766699
55  United States  2015   96

Couldn't find a good SQL source to derive the data I wanted so I replaced it with a second API call as you said we could do in Discord.

In [161]:
#SECOND API SOURCE AS I COULD NOT FIND A FREE SQL SOURCE!
api_key = 'ADNJHTYMA0SOHPS0'

# Construct the API endpoint URL
base_url = 'https://www.alphavantage.co/query'
params = {
    'function': 'TIME_SERIES_DAILY',
    'symbol': 'NDAQ',
    'apikey': api_key,
    'outputsize': 'full',
    'datatype': 'json'
}

# Make the API request
response = requests.get(base_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data2 = response.json()

    # Check if the response contains the expected data key
    if 'Time Series (Daily)' in data2:
        # Extract the time series data
        time_series_data = data2['Time Series (Daily)']

        # Convert the extracted data into a pandas DataFrame
        df_nq = pd.DataFrame(time_series_data).T  # Transpose to have dates as rows

        # Rename columns
        df_nq = df_nq.rename(columns={
            '1. open': 'Open',
            '2. high': 'High',
            '3. low': 'Low',
            '4. close': 'Close',
            '5. volume': 'Volume'
        })
        # Print the DataFrame
        print(df_nq)
    else:
        print("Error: 'Time Series (Daily)' not found in API response")
else:
    print(f"Error: {response.status_code} - {response.text}")


               Open     High      Low    Close   Volume
2024-04-05  61.0500  62.0600  60.9500  62.0400  1929386
2024-04-04  62.0000  62.4000  60.8400  61.0600  2076159
2024-04-03  61.4000  61.9900  61.3000  61.5800  2250746
2024-04-02  61.7500  61.8100  60.8650  61.3300  3135949
2024-04-01  62.9700  63.0400  62.2100  62.2900  2287324
...             ...      ...      ...      ...      ...
2002-07-08  14.0000  14.0000  14.0000  14.0000      300
2002-07-05  15.0000  15.0000  15.0000  15.0000      500
2002-07-03  14.0000  16.0000  14.0000  15.0000    10200
2002-07-02  15.5000  16.5000  15.0000  15.1300     7000
2002-07-01  15.0000  15.0000  15.0000  15.0000     1300

[5479 rows x 5 columns]


In [162]:
# Convert all columns to numeric type
df_nq = df_nq.apply(pd.to_numeric, errors='coerce')

# Convert the index to datetime
df_nq.index = pd.to_datetime(df_nq.index)

# Group by year and calculate yearly changes for each column
df_nq_year = df_nq.resample('Y').last().pct_change()

# Drop the first row (NaN values since there's no previous year)
df_nq_year = df_nq_year.drop(df_nq_year.index[0])

# Extract only the year from the index and reset index
df_nq_year.reset_index(inplace=True)
df_nq_year['Year'] = df_nq_year['index'].dt.strftime('%Y')
df_nq_year.drop(columns=['index'], inplace=True)

# Convert the values to decimal format
df_nq_year.iloc[:, :5] *= 100

# Print the resulting DataFrame
print(df_nq_year)


          Open        High         Low       Close       Volume  Year
0   -13.425926  -12.037037   -8.000000   -5.500000   -71.472393  2003
1     9.625668   10.526316   10.326087    7.936508  1031.182796  2004
2   245.365854  239.142857  243.842365  244.901961   627.756654  2005
3   -11.016949  -11.345128  -12.464183  -12.478681   130.172414  2006
4    56.984127   57.776370   61.243863   60.734005   -17.727840  2007
5   -52.295248  -50.050191  -52.415753  -50.070721    29.721341  2008
6   -14.158542  -18.408360  -15.443686  -19.789559   -11.442548  2009
7    17.037037   17.733990   19.298688   19.727548   -47.259081  2010
8     3.881857    3.347280    3.658279    3.286979   -27.743625  2011
9     1.056052    1.315789    0.983272    1.958384    12.305026  2012
10   60.610932   60.439560   60.064240   59.263705   -22.053872  2013
11   22.322322   21.743462   21.032372   20.502513   -20.635349  2014
12   20.233224   20.826514   21.313869   21.288574    83.223041  2015
13   14.514208   14.

In [163]:
# Convert 'Year' column in df_pop to int64
df_pop['Year'] = df_pop['Year'].astype(int)
us_data['Year'] = us_data['Year'].astype(int)
df_nq_year['Year'] = df_nq_year['Year'].astype(int)


years_df = pd.DataFrame({'Year': range(2013, 2022)})

# Merge the new DataFrame with 'df_pop' to get corresponding population data
merged_df = pd.merge(years_df, df_pop, on='Year', how='left')

# Merge 'us_data' with 'merged_df' to add debt percent values
merged_df = pd.merge(merged_df, us_data, on=['Year'], how='left')

# Merge 'df_nq_year' with 'merged_df' to add debt percent values
merged_df = pd.merge(merged_df, df_nq_year, on=['Year'], how='left')

# Rename the 'Value' column to 'Debt_Percent'
merged_df = merged_df.rename(columns={'Value': 'US Debt Percent'})

columns_to_drop3 = ['Country Name', 'High', 'Low', 'Volume']
merged_df = merged_df.drop(columns=columns_to_drop3)

merged_df = merged_df.rename(columns={'Open': 'Open Nasdaq % Change'})
merged_df = merged_df.rename(columns={'Close': 'Close Nasdaq % Change'})

# Print the resulting DataFrame
print(merged_df)


   Year         Nation  Population  US Debt Percent  Open Nasdaq % Change  \
0  2013  United States   311536594        95.534688             60.610932   
1  2014  United States   314107084        95.766699             22.322322   
2  2015  United States   316515021        96.425228             20.233224   
3  2016  United States   318558162        98.504102             14.514208   
4  2017  United States   321004407        97.692936             14.115899   
5  2018  United States   322903030        99.060976              5.364583   
6  2019  United States   324697795       100.805042             32.278794   
7  2020  United States   326569308       126.243301             21.449925   
8  2021  United States   329725481       120.361476             60.769231   

   Close Nasdaq % Change  
0              59.263705  
1              20.502513  
2              21.288574  
3              15.385938  
4              14.466627  
5               6.169465  
6              31.298271  
7            

In [164]:
import sqlite3
# Create a SQLite database connection
conn = sqlite3.connect('project_database.db')
cursor = conn.cursor()

#drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS population")
cursor.execute("DROP TABLE IF EXISTS stock_data")
cursor.execute("DROP TABLE IF EXISTS debt_data")

#rename in order to to_sql to work later on
us_data.rename(columns={'Country Name': 'Country_Name'}, inplace=True)

#generate the tables
cursor.execute('''CREATE TABLE population(
                  Population INTEGER PRIMARY KEY,
                  Nation TEXT,
                  Year INTEGER
                )''')

cursor.execute('''CREATE TABLE stock_data(
                  Open FLOAT PRIMARY KEY,
                  Year INTEGER,
                  Close FLOAT,
                  High FLOAT,
                  Low FLOAT,
                  Volume FLOAT
                )''')

cursor.execute('''CREATE TABLE debt_data(
                  Value FLOAT PRIMARY KEY,
                  Country_Name TEXT,
                  Year INTEGER
                )''')

# Convert DataFrames to SQL tables (assuming these DataFrames are defined elsewhere in your code)
df_pop.to_sql('population', conn, index=False, if_exists='append')
df_nq_year.to_sql('stock_data', conn, index=False, if_exists='append')
us_data.to_sql('debt_data', conn, index=False, if_exists='append')


#create the other tables from scratch
df_nq.to_sql('stock_daily_data', conn, index=False, if_exists='replace')
merged_df.to_sql('Combined_data', conn, index=False, if_exists='replace')

cursor.execute("DROP TABLE IF EXISTS Combined_with_fk")

# Enable foreign key constraints
cursor.execute("PRAGMA foreign_keys=ON")

# Define the schema for Combined_with_fk table
cursor.execute('''CREATE TABLE Combined_with_fk(
                  Year INTEGER PRIMARY KEY,
                  Population INTEGER,
                  OpenNasdaq FLOAT,
                  Debt FLOAT,
                  FOREIGN KEY(Population) REFERENCES population(Population),
                  FOREIGN KEY(OpenNasdaq) REFERENCES stock_data(Open),
                  FOREIGN KEY(Debt) REFERENCES debt_data(Value)
                )''')

# Populate Combined_with_fk
for year in range(2013, 2023):
    cursor.execute('''
        INSERT INTO Combined_with_fk (Year, Population, OpenNasdaq, Debt)
        SELECT ?, P.Population, S.Open, D.Value
        FROM population P
        LEFT JOIN stock_data S ON P.Year = S.Year
        LEFT JOIN debt_data D ON P.Year = D.Year
        WHERE P.Year = ?
    ''', (year, year))

# Commit the changes and close the connection
conn.commit()
conn.close()


In [165]:
import sqlite3

# Create a SQLite database connection
conn = sqlite3.connect('project_database.db')
cursor = conn.cursor()

# Fetch table names excluding 'stock_daily_data'
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name != 'stock_daily_data';")
table_names = cursor.fetchall()

# Iterate over table names and display data
for table in table_names:
    table_name = table[0]
    print(f"Table: {table_name}")
    # Fetch and display data from the table
    cursor.execute(f"SELECT * FROM {table_name};")
    data = cursor.fetchall()
    for row in data:
        print(row)
    print()

# Close the connection
conn.close()


Table: Year
(2013,)
(2014,)
(2015,)
(2016,)
(2017,)
(2018,)
(2019,)
(2020,)
(2021,)

Table: population
(311536594, 'United States', 2013)
(314107084, 'United States', 2014)
(316515021, 'United States', 2015)
(318558162, 'United States', 2016)
(321004407, 'United States', 2017)
(322903030, 'United States', 2018)
(324697795, 'United States', 2019)
(326569308, 'United States', 2020)
(329725481, 'United States', 2021)

Table: stock_data
(-13.425925925925931, 2003, -5.500000000000005, -12.037037037037045, -8.000000000000007, -71.47239263803681)
(9.625668449197855, 2004, 7.9365079365079305, 10.526315789473696, 10.326086956521753, 1031.1827956989248)
(245.3658536585366, 2005, 244.90196078431376, 239.14285714285714, 243.84236453201967, 627.7566539923955)
(-11.016949152542367, 2006, -12.478681068789088, -11.345127773097442, -12.464183381088823, 130.17241379310346)
(56.984126984127, 2007, 60.734004546930834, 57.77636997149194, 61.24386252045826, -17.72784019975031)
(-52.29524772497472, 2008, -50

In [166]:
conn = sqlite3.connect('project_database.db')
cursor = conn.cursor()

#query the database to find the name of the contry (USA), population and debt percentage for an individual year
cursor.execute("SELECT p.Nation, p.Population, d.Value AS Debt FROM population p INNER JOIN debt_data d ON p.Year = d.Year WHERE p.Year = 2019;")

#print this result
results = cursor.fetchall()
for row in results:
    print(row)
# Close the connection
conn.close()

('United States', 324697795, 100.805042202199)


In [167]:
conn = sqlite3.connect('project_database.db')
cursor = conn.cursor()

#query the database to select the average Debt value and Open value for every year
cursor.execute("SELECT p.Year, AVG(d.Value) AS AvgDebt, AVG(s.Open) AS AvgOpenNasdaq FROM population p LEFT JOIN debt_data d ON p.Year = d.Year LEFT JOIN stock_data s ON p.Year = s.Year GROUP BY p.Year ORDER BY p.Year;")

results = cursor.fetchall()
for row in results:
    print(row)
# Close the connection
conn.close()

(2013, 95.5346882113219, 60.61093247588425)
(2014, 95.7666986884038, 22.322322322322318)
(2015, 96.4252279492666, 20.233224222585932)
(2016, 98.5041017174837, 14.514207929215583)
(2017, 97.6929363628875, 14.115898959881124)
(2018, 99.0609761173397, 5.364583333333339)
(2019, 100.805042202199, 32.27879387048937)
(2020, 126.243300547564, 21.449925261584447)
(2021, 120.36147560281, 60.769230769230774)


In [168]:
conn = sqlite3.connect('project_database.db')
cursor = conn.cursor()

#Query the database for the average population, debt, open nasdaq value and the debt percent to nasdaq percent change ratio
cursor.execute("SELECT p.Year, AVG(p.Population) AS AvgPopulation, AVG(d.Value) AS AvgDebt, AVG(s.Open) AS AvgOpenNasdaq, AVG(d.Value) / AVG(s.Open) AS DebtToNasdaqRatio FROM population p LEFT JOIN debt_data d ON p.Year = d.Year LEFT JOIN stock_data s ON p.Year = s.Year GROUP BY p.Year ORDER BY p.Year;")

results = cursor.fetchall()
for row in results:
    print(row)
# Close the connection
conn.close()

(2013, 311536594.0, 95.5346882113219, 60.61093247588425, 1.5761956516562923)
(2014, 314107084.0, 95.7666986884038, 22.322322322322318, 4.2901763224087635)
(2015, 316515021.0, 96.4252279492666, 20.233224222585932, 4.7656877069364505)
(2016, 318558162.0, 98.5041017174837, 14.514207929215583, 6.786736293008814)
(2017, 321004407.0, 97.6929363628875, 14.115898959881124, 6.920773281286664)
(2018, 322903030.0, 99.0609761173397, 5.364583333333339, 18.465735353911846)
(2019, 324697795.0, 100.805042202199, 32.27879387048937, 3.122949469755721)
(2020, 326569308.0, 126.243300547564, 21.449925261584447, 5.8854890638550765)
(2021, 329725481.0, 120.36147560281, 60.769230769230774, 1.9806318770082658)
